In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly as py
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot # plotly offline mode
init_notebook_mode(connected=True) 
import plotly.graph_objs as go
from wordcloud import WordCloud
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import missingno as msno
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.listdir("../input")

In [ ]:
df_train1 = pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv')
df_train2 = pd.read_csv('/kaggle/input/corona-virus-report/country_wise_latest.csv')
df_train3 = pd.read_csv('/kaggle/input/corona-virus-report/day_wise.csv')
df_train4 = pd.read_csv('/kaggle/input/corona-virus-report/usa_county_wise.csv')
df_train5 = pd.read_csv('/kaggle/input/corona-virus-report/worldometer_data.csv')
df_train6 = pd.read_csv('/kaggle/input/corona-virus-report/full_grouped.csv')

In [ ]:
df_train1.columns
df_train2.columns
df_train3.columns
df_train4.columns
df_train5.columns
df_train6.columns


In [ ]:
df_train1.describe()

In [ ]:
df_train1.isnull().sum() / df_train1.shape[0]


In [ ]:
df_train2.isnull().sum() / df_train2.shape[0]

In [ ]:
df_train3.isnull().sum() / df_train3.shape[0]

In [ ]:
df_train4.isnull().sum() / df_train4.shape[0]

In [ ]:
df_train5.isnull().sum() / df_train5.shape[0]

In [ ]:
df_train6.isnull().sum() / df_train6.shape[0]

In [ ]:
df_train1.info()

In [ ]:
full_grouped = pd.read_csv('../input/corona-virus-report/full_grouped.csv')
full_grouped['Date'] = pd.to_datetime(full_grouped['Date'])
day_wise = pd.read_csv('../input/corona-virus-report/day_wise.csv')
day_wise['Date'] = pd.to_datetime(day_wise['Date'])
country_wise = pd.read_csv('../input/corona-virus-report/country_wise_latest.csv')
country_wise = country_wise.replace('', np.nan).fillna(0)
covid_1 = pd.read_csv('../input/corona-virus-report/full_grouped.csv')
covid5 = covid_1.groupby(['Date'])['New cases'].sum().reset_index()
covid5['Rolling Ave.'] = covid5.rolling(window=7).mean()   

In [ ]:
data = pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv')

In [ ]:
data.corr()

f,ax=plt.subplots(figsize=(12,12))
sns.heatmap(data.corr(),annot=True,annot_kws={"size":12},linewidths=1.0,fmt='.2f',ax=ax)
plt.title("Correlations",fontsize=20)
plt.show

In [ ]:
data_last = data.tail(1)
data_last_day = data[data["Date"] == data_last["Date"].iloc[0]] 
country_list = list(data_last_day["Country/Region"].unique())
confirmed = []
deaths = []
recovered = []
active = []
for i in country_list:
    x = data_last_day[data_last_day["Country/Region"] == i]
    confirmed.append(sum(x["Confirmed"]))
    deaths.append(sum(x["Deaths"]))
    recovered.append(sum(x["Recovered"]))
    active.append(sum(x["Active"]))
data_maps = pd.DataFrame(list(zip(country_list,confirmed,deaths,recovered,active)),columns = ["Country/Region","Confirmed","Deaths","Recovered","Active"])
data_maps.head()

In [ ]:
grp = data_maps.groupby(["Country/Region"])["Recovered"].max()
grp=grp.reset_index()
fig = px.choropleth(grp,locations ="Country/Region", 
                    color=np.log10(grp["Recovered"]), 
                    hover_name=grp["Country/Region"],
                    hover_data=["Recovered"],
                    color_continuous_scale=px.colors.sequential.Plasma,locationmode = "country names")
fig.update_geos(fitbounds ="locations",)
fig.update_layout(title_text="Recovered cases on world map")
fig.update_coloraxes(colorbar_title="Recovered Cases",colorscale="Greens")
fig.show()


In [ ]:
def plot_hbar(df, col, n, hover_data=[]):
    fig = px.bar(df.sort_values(col).tail(n), 
                 x=col, y="Country/Region", color='WHO Region',  
                 text=col, orientation='h', width=700, hover_data=hover_data,
                 color_discrete_sequence = px.colors.qualitative.Dark2)
    fig.update_layout(title=col, xaxis_title="", yaxis_title="", 
                      yaxis_categoryorder = 'total ascending',
                      uniformtext_minsize=8, uniformtext_mode='hide')
    fig.show()

In [ ]:
plot_hbar(country_wise, 'Deaths / 100 Cases', 30)

In [ ]:
plot_hbar(country_wise, 'Recovered / 100 Cases', 30)

In [ ]:
country_wise = pd.read_csv('../input/corona-virus-report/country_wise_latest.csv')
country_wise = country_wise.replace('', np.nan).fillna(0)

In [ ]:
def plot_map(df, col, pal):
    df = df[df[col]>0]
    fig = px.choropleth(df, locations="Country/Region", locationmode='country names', 
                  color=col, hover_name="Country/Region", 
                  title=col, hover_data=[col], color_continuous_scale=pal)
#     fig.update_layout(coloraxis_showscale=False)
    fig.show()

In [ ]:
country_wise.isnull().sum() / country_wise.shape[0]

In [ ]:
plot_map(country_wise, 'Active', 'matter')

In [ ]:
plot_map(country_wise, 'Deaths / 100 Recovered', 'matter')

In [ ]:
plot_map(country_wise, '1 week % increase', 'matter')

In [ ]:

fig = px.choropleth(full_grouped, locations="Country/Region", 
                    color=np.log(full_grouped["Confirmed"]),
                    locationmode='country names', hover_name="Country/Region", 
                    animation_frame=full_grouped["Date"].dt.strftime('%Y-%m-%d'),
                    title='Animation', color_continuous_scale=px.colors.sequential.matter)
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [ ]:
plot_map(country_wise, 'Recovered / 100 Cases', 'matter')

In [ ]:
plot_map(country_wise, 'Recovered / 100 Cases', 'matter')

In [ ]:
plot_map(country_wise, 'Confirmed', 'matter')

In [ ]:
df_train6.describe()

In [ ]:
choro_map=px.choropleth(full_grouped, 
                    locations="Country/Region", 
                    locationmode = "country names",
                    color="Confirmed", 
                    animation_frame=full_grouped["Date"].dt.strftime('%Y-%m-%d')
                   )
choro_map.update_layout(
    title_text = 'Coronavirus Animation',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
choro_map.show()

In [ ]:
#group = all.groupby(['Date', 'Country/Region'])['Confirmed'].sum()
#group = group.reset_index()
#group.head()

In [ ]:
Circle=country_wise[country_wise.Confirmed>100000]
labels=Circle['Country/Region']
confirmed = Circle['Confirmed']
fig ={"data":[{"values":confirmed,"domain":{"x":[0,.7]},"name":"Number of confirmed cases","hoverinfo":"label+value","hole":.1,"type":"pie"},],
    "layout":{"title": "Circle of covid 19","annotations":[{"font":{"size":30},"showarrow":False,"text":"percentage of Confirmed","x":0.20,"y":1.2},]}}
iplot(fig)

In [ ]:
country_wise = pd.read_csv('../input/corona-virus-report/country_wise_latest.csv')
day_wise=pd.read_csv("../input/corona-virus-report/day_wise.csv")
country_wise = country_wise.replace('', np.nan).fillna(0)

In [ ]:
sortedcountrywise=country_wise.sort_values('Confirmed',ascending=False)
top20sortedcountrywise=sortedcountrywise.iloc[:20,:]
normalizedconfirmed=top20sortedcountrywise['Confirmed']/100000
normalizeddeath=top20sortedcountrywise['Deaths']/10000

In [ ]:
trace1=go.Bar( x=top20sortedcountrywise['Country/Region'],y=normalizedconfirmed,name="Confirmed cases 100k",marker=dict(color='rgba(10,10,200,0.6)'),text=top20sortedcountrywise['Country/Region'],)
trace2=go.Bar( x=top20sortedcountrywise['Country/Region'],y=normalizeddeath,name="Number of death 10k",marker=dict(color='rgba(250,10,10,0.6)'),text=top20sortedcountrywise['Country/Region'],)
data=[trace1,trace2]
layout=dict(title="Confirmed cases and deaths in countries", xaxis=dict(title='Countries',ticklen=5,zeroline=False))
fig=dict(data=data,layout=layout)
iplot(fig)

In [ ]:
bar_data=go.Bar(x=covid5['Date'],y=covid5['New cases'],name='Daily New Cases',marker=dict(color='red'),hoverinfo='text',hovertext='<b>Date</b>:'+covid5['Date'].astype(str)+'<br>')
line_data=go.Scatter(x=covid5['Date'],y=covid5['Rolling Ave.'],name='Rolling Ave. of the last 7 days',mode='lines',marker=dict(color='#000080'),
hoverinfo='text',hovertext='<b>Date</b>:'+covid5['Date'].astype(str)+'<br>'+'<b>Rolling Ave<b/>'+['f{x:,.0f}' for x in covid5['Rolling Ave.']]+'<br>')
data=[bar_data,line_data]



In [ ]:
country_list=list(full_grouped['Country/Region'])
f,ax = plt.subplots(figsize = (9,150))
sns.barplot(x=full_grouped.Confirmed,y=country_list,color='green',alpha = 0.5,label='Confirmed cases' )
sns.barplot(x=full_grouped.Deaths ,y=country_list,color='red',alpha = 0.5,label='number of deaths' )
sns.barplot(x=full_grouped.Recovered ,y=country_list,color='blue',alpha = 0.5,label='recovered people' )
sns.barplot(x=full_grouped.Active ,y=country_list,color='orange',alpha = 0.5,label='active cases' )

ax.legend(loc='lower right',frameon = True) 
ax.set(xlabel='Number of cases', ylabel='Countries',title = "The number of Covid-19 cases  ")
plt.show()

In [ ]:
trace1= go.Scatter(x=day_wise.Date,y=day_wise.Confirmed,mode="lines",name="Total Confirmed cases",marker=dict(color='rgba(10,10,200,0.6)'),text= day_wise.Confirmed ,)
trace2= go.Scatter(x=day_wise.Date,y=day_wise.Active, mode="lines",name="Active cases",marker=dict(color='rgba(100,200,100,0.6)'),text= day_wise.Confirmed ,)
                   
data=[trace1,trace2]
layout=dict(title ="Totla Confirmed cases in the World", xaxis=dict(title ='Dates',ticklen=10,zeroline=False))
fig=dict(data=data,layout=layout)
iplot(fig)                  
    

In [ ]:
bar_data=go.Bar(x=covid5['Date'],y=covid5['New cases'],name='Daily New Cases',marker=dict(color= 'red'),hoverinfo='text',hovertext='<b>Date</b>:'+covid5['Date'].astype(str)+'<br>'+'<b>New Cases</b>:'+[f'{x:,.0f}' for x in covid5['New cases']]+'<br>')
#line_data=go.Scatter(x=covid5['Date'],y=covid5['Rolling Ave.'],name='Rolling Ave. of the last 7 days', mode ='lines',marker=dict(color='#000080'),hoverinfo='text',hovertest='<b>Date</b>:'+covid5['Date'].astype(str)+'<br>'+'<b>Rolling Ave<b/>:'+[f'{x:,.0f}' for x in covid5['Rollong Ave']]+'<br>')
line_data = go.Scatter(x=covid5['Date'],y=covid5['Rolling Ave.'],name='Rolling Ave. of the last 7 days',mode='lines',marker=dict(color='#000080'),hoverinfo='text',hovertext='<b>Date</b>: ' + covid5['Date'].astype(str) + '<br>' +'<b>Rolling Ave</b>: ' + [f'{x:,.0f}' for x in covid5['Rolling Ave.']] + '<br>'    )
data=[bar_data,line_data]
layout =go.Layout(title={'text':'Daily New Casas','y':0.93,'x':0.43,'xanchor':'center','yanchor':'top'},titlefont={'family':'Oswald','color':'rgb(230,34,144)','size':25},
hovermode='x',
xaxis=dict(title='<b>Date</b>',color='rgb(230,34,144)',showline=True,showgrid=True,showticklabels=True,linecolor='rgb(104,204,104)',linewidth=2,ticks='outside',tickfont=dict(family='Arial',size=12,color='rgb(17,37,239)')),
yaxis=dict(title='<b>date</b>',color='rgb(230,34,144)',showline=True,showgrid=True,showticklabels=True,linecolor='rgb(04,204,104)',linewidth=2, ticks='outside',tickfont=dict(family='Arial',size=12,color='rgb(17,37,239)')),
#legend=dict(title='',x=0.25,y=1.08,orientation='h',bgcolor='rgba(255,255,255,0)',traceorder="normal",font=dict(family="sans-serif",size=12,color='#000000')),)    
legend=dict(title='',x=0.25,y=1.08,orientation='h',bgcolor='rgba(255,255,255,0)',traceorder="normal",font=dict(family="sans-serif",size=12,color='#000000')),)

figure=go.Figure(data=data,layout=layout)
figure.show()